In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.neural_network import MLPClassifier
import torch
import torch.nn as nn
import torch.optim as optim

## Load Dataset

In [3]:
df = pd.read_csv("C:/Users/HP/Desktop/DERSLER/Derin Öğrenme Lab/lab3_12032025/BankNote_Authentication.csv")
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
X, y = df.iloc[:, :-1], df.iloc[:, -1]
X = X.to_numpy()
y = y.to_numpy().reshape(-1, 1)

#### Split and Normalize Data 

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Scikit-learn MLPClassifier Model

In [5]:
mlp_sklearn = MLPClassifier(hidden_layer_sizes=(5, 5), activation='relu', solver='sgd', max_iter=400, random_state=42)
mlp_sklearn.fit(X_train, y_train.ravel())
y_pred_sklearn = mlp_sklearn.predict(X_test)
accuracy_sklearn = accuracy_score(y_test, y_pred_sklearn)
conf_matrix_sklearn = confusion_matrix(y_test, y_pred_sklearn)
report_sklearn = classification_report(y_test, y_pred_sklearn)

In [6]:
print("\nScikit-learn MLP Model:")
print(f"Accuracy: {accuracy_sklearn:.4f}")
print("Confusion Matrix:")
print(conf_matrix_sklearn)
print("Classification Report:")
print(report_sklearn)


Scikit-learn MLP Model:
Accuracy: 0.9927
Confusion Matrix:
[[151   2]
 [  0 122]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.99      0.99       153
           1       0.98      1.00      0.99       122

    accuracy                           0.99       275
   macro avg       0.99      0.99      0.99       275
weighted avg       0.99      0.99      0.99       275



## PyTorch Model 

In [7]:
class MLP_PyTorch(nn.Module):
    def __init__(self, input_size, hidden_layers, output_size=1):
        super(MLP_PyTorch, self).__init__()
        layers = []
        prev_size = input_size
        for hidden_size in hidden_layers:
            layers.append(nn.Linear(prev_size, hidden_size))
            layers.append(nn.ReLU())
            prev_size = hidden_size
        layers.append(nn.Linear(prev_size, output_size))
        layers.append(nn.Sigmoid())
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

In [8]:
# Initialize Model
torch.manual_seed(42)
model_torch = MLP_PyTorch(input_size=X.shape[1], hidden_layers=[5, 5])
criterion = nn.BCELoss()
optimizer = optim.SGD(model_torch.parameters(), lr=0.1)

In [9]:
X_train_torch = torch.tensor(X_train, dtype=torch.float32)
y_train_torch = torch.tensor(y_train, dtype=torch.float32)
X_test_torch = torch.tensor(X_test, dtype=torch.float32)
y_test_torch = torch.tensor(y_test, dtype=torch.float32)

### Train PyTorch Model

In [10]:
for epoch in range(400):
    optimizer.zero_grad()
    outputs = model_torch(X_train_torch).squeeze()
    loss = criterion(outputs, y_train_torch.squeeze())
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 50 == 0:
        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

Epoch 50, Loss: 0.6526
Epoch 100, Loss: 0.4628
Epoch 150, Loss: 0.2599
Epoch 200, Loss: 0.1667
Epoch 250, Loss: 0.1224
Epoch 300, Loss: 0.0974
Epoch 350, Loss: 0.0814
Epoch 400, Loss: 0.0703


In [11]:
# Evaluate PyTorch Model
y_pred_torch = (model_torch(X_test_torch).detach().numpy().squeeze() >= 0.5).astype(int)
accuracy_torch = accuracy_score(y_test, y_pred_torch)
conf_matrix_torch = confusion_matrix(y_test, y_pred_torch)
report_torch = classification_report(y_test, y_pred_torch)

In [12]:
print("\nPyTorch MLP Model:")
print(f"Accuracy: {accuracy_torch:.4f}")
print("Confusion Matrix:")
print(conf_matrix_torch)
print("Classification Report:")
print(report_torch)


PyTorch MLP Model:
Accuracy: 0.9927
Confusion Matrix:
[[151   2]
 [  0 122]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.99      0.99       153
           1       0.98      1.00      0.99       122

    accuracy                           0.99       275
   macro avg       0.99      0.99      0.99       275
weighted avg       0.99      0.99      0.99       275

